# 1. INSTALL 

## 1.A conda env & Jupyter

```
conda create --name jadc_nircam python=3.9
conda activate jadc_nircam
conda install jupyter 
# delete with:  conda env remove --name jadc_nircam
```

## 1.B jupyter and grizli install
```
pip install "grizli[jwst]"
pip install pyregion
pip install git+https://github.com/gbrammer/dust_attenuation.git
pip install git+https://github.com/gbrammer/dust_extinction.git 
```

## 1.C put these into the ~/.bashrc or equivalent
```
export GRIZLI="${HOME}/grizli" # dir for config files, here or anywhere else 
export iref="${GRIZLI}/iref/"  # for WFC3 calibration files
export jref="${GRIZLI}/jref/"  # for ACS calibration files
mkdir $GRIZLI
mkdir $GRIZLI/CONF      # needed for grism configuration files
mkdir $GRIZLI/templates # for redshift fits
mkdir $iref
mkdir $jref
```

In [2]:
# Grizli and requirements, e.g., on google collab
import importlib
import sys
import os

import grizli

## Module versions
print(sys.version + '\n')
for module in ['grizli', 'tristars', 'mastquery', 'sep',
               'numpy', 'scipy', 'astropy', 'astroquery', 
               'shapely', 'drizzlepac','jwst']:
    try:
        mod = importlib.import_module(module)
        print('{0:>20} : {1}'.format(module, mod.__version__))
    except ImportError:
        print('{0:>20} : {1}'.format(module, '*failed*'))

3.9.15 (main, Nov 24 2022, 08:29:02) 
[Clang 14.0.6 ]

              grizli : 1.7
            tristars : 0.1
           mastquery : 1.5.1
                 sep : 1.2.1
               numpy : 1.23.5
               scipy : 1.9.3
             astropy : 5.1.1
          astroquery : 0.4.6
             shapely : 1.8.5.post1


The following task in the stsci.skypac package can be run with TEAL:
                                    skymatch                                    
The following tasks in the drizzlepac package can be run with TEAL:
    astrodrizzle       config_testbed      imagefindpars           mapreg       
       photeq            pixreplace           pixtopix            pixtosky      
  refimagefindpars       resetbits          runastrodriz          skytopix      
     tweakback            tweakreg           updatenpol
          drizzlepac : 3.5.0
                jwst : 1.8.4


In [3]:
import jwst

if os.getenv('CRDS_PATH') is None:
    cache_dir = os.path.join(os.getcwd(), 'crds_cache')
    print('Set CRDS_PATH=',cache_dir)
    os.environ['CRDS_PATH'] = cache_dir
    if not os.path.exists(cache_dir):
        os.makedirs(cache_dir)
else:
    print("CRDS_PATH={",os.getenv('CRDS_PATH'),'}')

if os.getenv('CRDS_SERVER_URL') is None:
    server = 'https://jwst-crds.stsci.edu'
    print('Set CRDS_SERVER_URL={',server,'}')
    os.environ['CRDS_SERVER_URL'] = 'https://jwst-crds.stsci.edu'
else:
    print("CRDS_SERVER_URL={",os.getenv('CRDS_SERVER_URL'),"}")

CRDS_PATH={ /Users/ivo/Astro/PROJECTS/JWST/crds_cache }
CRDS_SERVER_URL={ https://jwst-crds.stsci.edu }


# 2. Download observations from MAST

In [19]:
from astroquery.mast import Observations
from astropy.table import unique, vstack, Table

In [40]:
# get F277W band data from UNCOVER program 2561
filters = ['F444W','F200W']
obs = [Observations.query_criteria(obs_collection = 'JWST', proposal_id = '2561', 
                                   productType='science',
                                   instrument_name = 'Nircam', filters = f)
                                   for f in filters ]
obs = vstack(obs)
obs

dataproduct_type,calib_level,obs_collection,obs_id,target_name,s_ra,s_dec,t_min,t_max,t_exptime,wavelength_region,filters,em_min,em_max,target_classification,obs_title,t_obs_release,instrument_name,proposal_pi,proposal_id,proposal_type,project,sequence_number,provenance_name,s_region,jpegURL,dataURL,dataRights,mtFlag,srcDen,intentType,obsid,objID
str5,int64,str4,str36,str31,float64,float64,float64,float64,float64,str8,str5,float64,float64,str1,str84,float64,str6,str10,str4,str2,str4,int64,str7,str111,str62,str63,str6,bool,float64,str7,str9,str9
image,3,JWST,jw02561-o003_t006_nircam_clear-f200w,ABELL2744-PREIMG-REPEAT1-19N47D,3.587564583333333,-30.35995277777778,59898.59822264086,59898.68309895833,53597.952,Infrared,F200W,1755.0,2226.0,--,UNCOVER: Ultra-deep NIRCam and NIRSpec Observations Before the Epoch of Reionization,59899.8116434,NIRCAM,"Labbe, Ivo",2561,GO,JWST,--,CALJWST,POLYGON 3.533472689 -30.342853699 3.56835268 -30.313121634 3.642172977 -30.377569012 3.607292447 -30.407320676,mast:JWST/product/jw02561-o003_t006_nircam_clear-f200w_i2d.jpg,mast:JWST/product/jw02561-o003_t006_nircam_clear-f200w_i2d.fits,PUBLIC,False,nan,science,101124318,188917710
image,3,JWST,jw02561-o001_t003_nircam_clear-f200w,ABELL2744-PREIMG,3.5760475,-30.37945555555558,59885.53356233993,59890.26014797454,160793.856,Infrared,F200W,1755.0,2226.0,--,UNCOVER: Ultra-deep NIRCam and NIRSpec Observations Before the Epoch of Reionization,59890.89464115,NIRCAM,"Labbe, Ivo",2561,GO,JWST,--,CALJWST,POLYGON 3.501657733 -30.37399924 3.560945726 -30.316498285 3.65083826 -30.38543215 3.591561352 -30.442973698,mast:JWST/product/jw02561-o001_t003_nircam_clear-f200w_i2d.jpg,mast:JWST/product/jw02561-o001_t003_nircam_clear-f200w_i2d.fits,PUBLIC,False,nan,science,100380644,188917735
image,3,JWST,jw02561-o003_t006_nircam_clear-f444w,ABELL2744-PREIMG-REPEAT1-19N47D,3.587564583333333,-30.35995277777778,59898.68757129942,59898.80140328703,16491.68,Infrared,F444W,3880.0,4986.0,--,UNCOVER: Ultra-deep NIRCam and NIRSpec Observations Before the Epoch of Reionization,59899.8116434,NIRCAM,"Labbe, Ivo",2561,GO,JWST,--,CALJWST,POLYGON 3.53374544 -30.342876319 3.567901705 -30.313395677 3.641598538 -30.376938111 3.6074423 -30.406437914,mast:JWST/product/jw02561-o003_t006_nircam_clear-f444w_i2d.jpg,mast:JWST/product/jw02561-o003_t006_nircam_clear-f444w_i2d.fits,PUBLIC,False,nan,science,101095956,188917653
image,3,JWST,jw02561-o001_t003_nircam_clear-f444w,ABELL2744-PREIMG,3.5760475,-30.37945555555558,59885.62440289665,59890.37869892361,49475.03999999999,Infrared,F444W,3880.0,4986.0,--,UNCOVER: Ultra-deep NIRCam and NIRSpec Observations Before the Epoch of Reionization,59890.89464115,NIRCAM,"Labbe, Ivo",2561,GO,JWST,--,CALJWST,POLYGON 3.501873815 -30.374067855 3.560267414 -30.316718691 3.650216664 -30.384835698 3.591835162 -30.442224868,mast:JWST/product/jw02561-o001_t003_nircam_clear-f444w_i2d.jpg,mast:JWST/product/jw02561-o001_t003_nircam_clear-f444w_i2d.fits,PUBLIC,False,nan,science,100380027,188917701


In [41]:
# unpack obs list into a Table with each entry corresponding to a unique file
t = [Observations.get_product_list(o) for o in obs]
files = unique(vstack(t), keys='productFilename')
files

obsID,obs_collection,dataproduct_type,obs_id,description,type,dataURI,productType,productGroupDescription,productSubGroupDescription,productDocumentationURL,project,prvversion,proposal_id,productFilename,size,parent_obsid,dataRights,calib_level
str9,str4,str5,str36,str64,str1,str69,str9,str28,str11,str1,str7,str5,str4,str51,int64,str9,str6,int64
100379049,JWST,image,jw02561001004_10101_00007_nrcalong,source/target (L3) : association generator,S,mast:JWST/product/jw02561-o001_20221118t004454_image2_00014_asn.json,INFO,--,ASN,--,CALJWST,1.8.2,2561,jw02561-o001_20221118t004454_image2_00014_asn.json,1795,100380027,PUBLIC,2
100379352,JWST,image,jw02561001004_10101_00006_nrcblong,source/target (L3) : association generator,S,mast:JWST/product/jw02561-o001_20221118t004454_image2_00018_asn.json,INFO,--,ASN,--,CALJWST,1.8.2,2561,jw02561-o001_20221118t004454_image2_00018_asn.json,1795,100380027,PUBLIC,2
100379152,JWST,image,jw02561001004_10101_00007_nrcblong,source/target (L3) : association generator,S,mast:JWST/product/jw02561-o001_20221118t004454_image2_00020_asn.json,INFO,--,ASN,--,CALJWST,1.8.2,2561,jw02561-o001_20221118t004454_image2_00020_asn.json,1795,100380027,PUBLIC,2
100379288,JWST,image,jw02561001004_10101_00008_nrcblong,source/target (L3) : association generator,S,mast:JWST/product/jw02561-o001_20221118t004454_image2_00024_asn.json,INFO,--,ASN,--,CALJWST,1.8.2,2561,jw02561-o001_20221118t004454_image2_00024_asn.json,1795,100380027,PUBLIC,2
100379011,JWST,image,jw02561001004_10101_00006_nrcalong,source/target (L3) : association generator,S,mast:JWST/product/jw02561-o001_20221118t004454_image2_00028_asn.json,INFO,--,ASN,--,CALJWST,1.8.2,2561,jw02561-o001_20221118t004454_image2_00028_asn.json,1795,100380027,PUBLIC,2
100379278,JWST,image,jw02561001004_10101_00003_nrcblong,source/target (L3) : association generator,S,mast:JWST/product/jw02561-o001_20221118t004454_image2_00037_asn.json,INFO,--,ASN,--,CALJWST,1.8.2,2561,jw02561-o001_20221118t004454_image2_00037_asn.json,1795,100380027,PUBLIC,2
100379148,JWST,image,jw02561001004_10101_00005_nrcblong,source/target (L3) : association generator,S,mast:JWST/product/jw02561-o001_20221118t004454_image2_00038_asn.json,INFO,--,ASN,--,CALJWST,1.8.2,2561,jw02561-o001_20221118t004454_image2_00038_asn.json,1795,100380027,PUBLIC,2
100379150,JWST,image,jw02561001004_10101_00002_nrcblong,source/target (L3) : association generator,S,mast:JWST/product/jw02561-o001_20221118t004454_image2_00044_asn.json,INFO,--,ASN,--,CALJWST,1.8.2,2561,jw02561-o001_20221118t004454_image2_00044_asn.json,1795,100380027,PUBLIC,2
100378998,JWST,image,jw02561001004_10101_00005_nrcalong,source/target (L3) : association generator,S,mast:JWST/product/jw02561-o001_20221118t004454_image2_00047_asn.json,INFO,--,ASN,--,CALJWST,1.8.2,2561,jw02561-o001_20221118t004454_image2_00047_asn.json,1795,100380027,PUBLIC,2


## 2.A First look at "level 3" STScI pipeline reduced coadded images 

In [ ]:
# only select "level3" reduced rectified 2D image (i2d.fits)
lvl3 = (files['calib_level'] == 3)
files_lvl3 = vstack( [f for f in files[lvl3] if 'i2d.fits' in f['dataURI']] )
files_lvl3

In [ ]:
# download only the first of the two files  
manifest = Observations.download_products(files_lvl3[0])

# add curl flag to get a download script
# manifest = Observations.download_products(files_lvl3, curl_flag=True)

In [ ]:
! find . 
# inspect the pipeline reduced files in ds9

## 2.B download level 2b calibrated frames

In [ ]:
# download level 2b products (calibrated individual frames ending in _cal.fits)
lvl2 = (files['calib_level'] == 2) & (files['productType'] == 'SCIENCE')
files_lvl2 = vstack( [f for f in files[lvl2] if 'cal.fits' in f['dataURI']] )

files_lvl2[0]

In [40]:
# download only the first 8 files (4 exposures of each a/b module)  
manifest = Observations.download_products(files_lvl2[0:8])

# Reduction of level 2b cal frames with Grizli

In [98]:
print(os.environ['CRDS_PATH'])
print(os.environ["CRDS_SERVER_URL"])
#os.environ['CRDS_CONTEXT'] = 'jwst_0863.pmap'
#print(os.environ['CRDS_CONTEXT'])

/Users/ivo/Astro/PROJECTS/JWST/crds_cache
https://jwst-crds.stsci.edu


In [14]:
# setup paths and files for grizli
from pathlib import Path
from glob import glob

import os
import grizli
from grizli import utils, prep, jwst_utils, multifit
from grizli.pipeline import auto_script
print('Grizli version: ', grizli.__version__)

# This rootname used throughout
root='uncover'            

# Some directories needed for grizli 
if 'HOME_PATH' not in locals(): HOME_PATH=os.getcwd()
working_dir = 'Prep'   # hardcoded in autoscript grizli... beware !
raw_dir = '../RAW'

Path(working_dir).mkdir(exist_ok=True)
os.chdir(HOME_PATH+'/'+working_dir)
Path(raw_dir).mkdir(exist_ok=True)
os.chdir(HOME_PATH)

# copy cal files into RAW directory
! find . -name "*cal.fits" -exec cp {} RAW \;

# download 60mas F814W image from MAST archive
# note: may want to use 30mas for better resolution
# https://archive.stsci.edu/pub/hlsp/frontier/abell2744/images/hst/v1.0-epoch2/hlsp_frontier_hst_acs-30mas_abell2744_f814w_v1.0-epoch2_drz.fits
! wget https://archive.stsci.edu/pub/hlsp/frontier/abell2744/images/hst/v1.0-epoch2/hlsp_frontier_hst_acs-60mas-selfcal_abell2744_f814w_v1.0-epoch2_drz.fits
! cp pub/hlsp/frontier/abell2744/images/hst/v1.0-epoch2/hlsp_frontier_hst_acs-60mas-selfcal_abell2744_f814w_v1.0-epoch2_drz.fits hst_abell2744_f814w.fits

Grizli version:  1.7
cp: RAW/jw02561001002_02101_00003_nrcalong_cal.fits and ./RAW/jw02561001002_02101_00003_nrcalong_cal.fits are identical (not copied).
cp: RAW/jw02561001002_02101_00002_nrcalong_cal.fits and ./RAW/jw02561001002_02101_00002_nrcalong_cal.fits are identical (not copied).
cp: RAW/jw02561001002_02101_00001_nrcalong_cal.fits and ./RAW/jw02561001002_02101_00001_nrcalong_cal.fits are identical (not copied).
cp: RAW/jw02561001002_02101_00004_nrcalong_cal.fits and ./RAW/jw02561001002_02101_00004_nrcalong_cal.fits are identical (not copied).
cp: RAW/jw02561001002_02101_00001_nrcblong_cal.fits and ./RAW/jw02561001002_02101_00001_nrcblong_cal.fits are identical (not copied).
cp: RAW/jw02561001002_02101_00004_nrcblong_cal.fits and ./RAW/jw02561001002_02101_00004_nrcblong_cal.fits are identical (not copied).
cp: RAW/jw02561001002_02101_00003_nrcblong_cal.fits and ./RAW/jw02561001002_02101_00003_nrcblong_cal.fits are identical (not copied).
cp: RAW/jw02561001002_02101_00002_nrcblon

In [15]:
# go into Prep working director, parse file list
os.chdir(os.path.join(HOME_PATH, working_dir))
os.getcwd()

files = glob(raw_dir+'/*_cal.fits*')
info = utils.get_flt_info(files=files)
info

FILE,FILTER,PUPIL,INSTRUME,DETECTOR,TARGNAME,DATE-OBS,TIME-OBS,EXPSTART,EXPTIME,PA_V3,RA_TARG,DEC_TARG,POSTARG1,POSTARG2
str43,str17,str5,str6,str8,str16,str10,str12,float64,float64,float64,float64,float64,float64,float64
jw02561001002_02101_00003_nrcalong_cal.fits,NRCA5-F277W-CLEAR,CLEAR,NIRCAM,NRCALONG,ABELL2744-PREIMG,2022-11-02,08:58:12.376,59885.37375435382,837.468,41.33649879566452,3.5760475,-30.37945555555558,nan,nan
jw02561001002_02101_00002_nrcalong_cal.fits,NRCA5-F277W-CLEAR,CLEAR,NIRCAM,NRCALONG,ABELL2744-PREIMG,2022-11-02,08:42:38.232,59885.36294250197,837.468,41.33780687139559,3.5760475,-30.37945555555558,nan,nan
jw02561001002_02101_00001_nrcalong_cal.fits,NRCA5-F277W-CLEAR,CLEAR,NIRCAM,NRCALONG,ABELL2744-PREIMG,2022-11-02,08:27:04.153,59885.35213140243,837.468,41.33863158684239,3.5760475,-30.37945555555558,nan,nan
jw02561001002_02101_00004_nrcalong_cal.fits,NRCA5-F277W-CLEAR,CLEAR,NIRCAM,NRCALONG,ABELL2744-PREIMG,2022-11-02,09:13:46.455,59885.38456545336,837.468,41.33514613828189,3.5760475,-30.37945555555558,nan,nan
jw02561001002_02101_00001_nrcblong_cal.fits,NRCB5-F277W-CLEAR,CLEAR,NIRCAM,NRCBLONG,ABELL2744-PREIMG,2022-11-02,08:27:04.153,59885.35213140243,837.468,41.33863158684239,3.5760475,-30.37945555555558,nan,nan
jw02561001002_02101_00004_nrcblong_cal.fits,NRCB5-F277W-CLEAR,CLEAR,NIRCAM,NRCBLONG,ABELL2744-PREIMG,2022-11-02,09:13:46.455,59885.38456545336,837.468,41.33514613828189,3.5760475,-30.37945555555558,nan,nan
jw02561001002_02101_00003_nrcblong_cal.fits,NRCB5-F277W-CLEAR,CLEAR,NIRCAM,NRCBLONG,ABELL2744-PREIMG,2022-11-02,08:58:12.312,59885.37375361308,837.468,41.33649879566526,3.5760475,-30.37945555555558,nan,nan
jw02561001002_02101_00002_nrcblong_cal.fits,NRCB5-F277W-CLEAR,CLEAR,NIRCAM,NRCBLONG,ABELL2744-PREIMG,2022-11-02,08:42:38.232,59885.36294250197,837.468,41.33780687139559,3.5760475,-30.37945555555558,nan,nan


In [17]:
# from file list create visit groups 
visits, all_groups, info = auto_script.parse_visits(field_root=root,
                                                    visit_split_shift=0,
                                                    files=files,
                                                    combine_same_pa=False,
                                                    RAW_PATH=raw_dir)

2022-12-02 12:55:58,919 - stpipe - WARNING - /Users/ivo/anaconda/envs/jadc_nircam/lib/python3.9/site-packages/astropy/wcs/wcs.py:725: FITSFixedWarning: 'obsfix' made the change 'Set OBSGEO-L to    63.263556 from OBSGEO-[XYZ].
Set OBSGEO-B to    28.494908 from OBSGEO-[XYZ].
Set OBSGEO-H to 1394727362.015 from OBSGEO-[XYZ]'.
  warnings.warn(

2022-12-02 12:55:58,935 - stpipe - WARNING - /Users/ivo/anaconda/envs/jadc_nircam/lib/python3.9/site-packages/astropy/wcs/wcs.py:725: FITSFixedWarning: 'obsfix' made the change 'Set OBSGEO-L to    63.281076 from OBSGEO-[XYZ].
Set OBSGEO-B to    28.491626 from OBSGEO-[XYZ].
Set OBSGEO-H to 1394836264.421 from OBSGEO-[XYZ]'.
  warnings.warn(

2022-12-02 12:55:58,951 - stpipe - WARNING - /Users/ivo/anaconda/envs/jadc_nircam/lib/python3.9/site-packages/astropy/wcs/wcs.py:725: FITSFixedWarning: 'obsfix' made the change 'Set OBSGEO-L to    63.299758 from OBSGEO-[XYZ].
Set OBSGEO-B to    28.488124 from OBSGEO-[XYZ].
Set OBSGEO-H to 1394952435.627 from OBSG

# (2022-12-02 01:55:58.849)

auto_script.parse_visits(**{'files': ['../RAW/jw02561001002_02101_00001_nrcalong_cal.fits', '../RAW/jw02561001002_02101_00001_nrcblong_cal.fits', '../RAW/jw02561001002_02101_00002_nrcalong_cal.fits', '../RAW/jw02561001002_02101_00002_nrcblong_cal.fits', '../RAW/jw02561001002_02101_00003_nrcalong_cal.fits', '../RAW/jw02561001002_02101_00003_nrcblong_cal.fits', '../RAW/jw02561001002_02101_00004_nrcalong_cal.fits', '../RAW/jw02561001002_02101_00004_nrcblong_cal.fits'], 'field_root': 'uncover', 'RAW_PATH': '../RAW', 'use_visit': True, 'combine_same_pa': False, 'combine_minexp': 2, 'is_dash': False, 'filters': ['F410M', 'F467M', 'F547M', 'F550M', 'F621M', 'F689M', 'F763M', 'F845M', 'F200LP', 'F350LP', 'F435W', 'F438W', 'F439W', 'F450W', 'F475W', 'F475X', 'F555W', 'F569W', 'F600LP', 'F606W', 'F622W', 'F625W', 'F675W', 'F702W', 'F775W', 'F791W', 'F814W', 'F850LP', 'G800L', 'F098M', 'F127M', 'F139M', 'F153M', 'F105W', 'F110W', 'F125W', 'F140W', 'F160W', 'G102', 'G1

In [11]:
from grizli.pipeline.auto_script import get_yml_parameters

# Read the default parameters that can be edited and passed en-masse to `auto_script.go`
kwargs = get_yml_parameters()

# Parameter lists
visit_prep_args = kwargs['visit_prep_args']
preprocess_args = kwargs['preprocess_args']

# Maximum shift for "tweakshifts" relative alignment
visit_prep_args['tweak_max_dist'] = 3.

# Will fit and subtract a SExtractor-like background to each visit
# filter size is bh x pixel_scale arcsec, which is then interpreted
# at the pixel scale of each intrument
visit_prep_args['imaging_bkg_params']  = {'bh': 256, 'bw': 256, 'fh': 3, 'fw': 3,
                                          'pixel_scale': 0.06, 'get_median': False}

# External alignment reference catalogs, searched in this order
# LS_DR9 is a good option for fairly high source density and sky coverage, deeper than PS1
# GAIA preferred of course, but source density isn't usually very high for
# extragalactic fields
visit_prep_args['reference_catalogs'] = ['PS1','GAIA','SDSS','WISE']

# we dont have a master reference file 
# Use the alignment catalog we made earlier, which will supercede `reference_catalogs`
# preprocess_args['master_radec'] = os.path.join(HOME_PATH, 'uncover.radec') # (with absolute path)


In [12]:
# use calibration context of images 
from astropy.io import fits
im = fits.open(glob('../RAW/*_cal.fits*')[0])
print('CRDS_CONTEXT: ', im[0].header['CRDS_CTX'])
os.environ['CRDS_CONTEXT'] = im[0].header['CRDS_CTX']
print(os.getcwd())

CRDS_CONTEXT:  jwst_1017.pmap
/Users/ivo/Astro/PROJECTS/UNCOVER/work/reduce/jadc/Prep


In [12]:
# This script will do all the preprocessing of all visits
os.chdir(os.path.join(HOME_PATH, working_dir))
auto_script.preprocess(field_root=root, HOME_PATH=HOME_PATH,
                       visit_prep_args=visit_prep_args, **preprocess_args)

# (2022-12-01 14:24:02.281)

auto_script.preprocess(**{'field_root': 'uncover', 'HOME_PATH': '/Users/ivo/Astro/PROJECTS/UNCOVER/work/reduce/jadc', 'PERSIST_PATH': None, 'min_overlap': 0.2, 'make_combined': False, 'catalogs': ['PS1', 'DES', 'NSC', 'SDSS', 'GAIA', 'WISE'], 'use_visit': True, 'master_radec': None, 'parent_radec': None, 'use_first_radec': False, 'skip_imaging': False, 'clean': True, 'skip_single_optical_visits': False, 'visit_prep_args': {'align_thresh': None, 'align_rms_limit': 2, 'align_mag_limits': [14, 24, 0.05], 'align_assume_close': False, 'align_transform': None, 'align_ref_border': 100, 'max_err_percentile': 99, 'align_min_flux_radius': 1.0, 'catalog_mask_pad': 0.05, 'match_catalog_density': False, 'skymethod': 'localmin', 'drizzle_params': {}, 'single_image_CRs': True, 'run_tweak_align': False, 'tweak_threshold': 1.5, 'tweak_fit_order': -1, 'tweak_max_dist': 3.0, 'tweak_n_min': 10, 'tweak_ref_exp': 0, 'align_simple': False, 'align_clip': 120, 'column_average': Tru

2022-12-02 01:24:03,735 - stpipe - WARNING - /Users/ivo/anaconda/envs/jwst_nircam/lib/python3.9/site-packages/grizli/jwst_utils.py:895: RuntimeWarning: divide by zero encountered in divide
  sn_mask = (sci - back)/err > thresholds[0]



exposure_oneoverf_correction: jw02561001002_02101_00001_nrcalong_cal.fits axis=1 deg_pix=64


2022-12-02 01:24:03,820 - stpipe - WARNING - /Users/ivo/anaconda/envs/jwst_nircam/lib/python3.9/site-packages/numpy/lib/nanfunctions.py:1095: RuntimeWarning: All-NaN slice encountered
  result = np.apply_along_axis(_nanmedian1d, axis, a, overwrite_input)



exposure_oneoverf_correction: jw02561001002_02101_00001_nrcalong_cal.fits apply to file
ENV CRDS_CONTEXT = jwst_1017.pmap
jwst_utils.img_with_flat: Flat already applied
jwst_utils.get_jwst_skyflat: /Users/ivo//Astro/SOFT/grizli/CONF/NircamSkyFlat/nrcalong-f277w-clear_skyflat_smooth.fits not found
ENV CRDS_CONTEXT = jwst_1017.pmap
jwst.assign_wcs.AssignWcsStep: /Users/ivo/Astro/PROJECTS/JWST/crds_cache/references/jwst/nircam/jwst_nircam_distortion_0156.asdf
# (2022-12-01 14:24:06.274)

jwst_utils.pipeline_model_wcs_header(**{'datamodel': <ImageModel(2048, 2048) from jw02561001002_02101_00001_nrcalong_cal.fits>, 'step': 64, 'degrees': [3, 4, 5, 5], 'lsq_args': {'jac': '2-point', 'bounds': (-inf, inf), 'method': 'trf', 'ftol': 1e-12, 'xtol': 1e-12, 'gtol': 1e-12, 'x_scale': 1.0, 'loss': 'soft_l1', 'f_scale': 1000.0, 'diff_step': 1e-06, 'tr_solver': None, 'tr_options': {}, 'jac_sparsity': None, 'max_nfev': 100, 'verbose': 0, 'kwargs': {}}, 'crpix': None, 'verbose': True, 'initial_header': 

...

make_SEP_catalog: sep version = 1.2.1
# (2022-12-01 14:27:50.475)
sep: Image AB zeropoint =  26.475
# (2022-12-01 14:27:50.594)
SEP: Get background {'bw': 32.0, 'bh': 32.0, 'fw': 3, 'fh': 3}


/Users/ivo/anaconda/envs/jwst_nircam/lib/python3.9/site-packages/grizli/prep.py:1697: RuntimeWarning: divide by zero encountered in divide
  err = 1/np.sqrt(wht_data)
/Users/ivo/anaconda/envs/jwst_nircam/lib/python3.9/site-packages/grizli/prep.py:1772: RuntimeWarning: divide by zero encountered in divide
  ratio = bkg.rms()/err


SEP: err_scale=0.048
   SEP: Extract...
    Done.
compute_SEP_auto_params: sep version = 1.2.1
compute_SEP_auto_params: autoparams=[2.5, <Quantity 0.35 arcsec>, 2.4, 3.8]; pixel_scale=0.06299535663233176; subpix=0; flux_radii=[0.2, 0.5, 0.9]


/Users/ivo/anaconda/envs/jwst_nircam/lib/python3.9/site-packages/grizli/prep.py:1878: RuntimeWarning: divide by zero encountered in log10
  tab['mag_iso'] = 23.9-2.5*np.log10(tab['flux_iso'])
/Users/ivo/anaconda/envs/jwst_nircam/lib/python3.9/site-packages/grizli/prep.py:1913: RuntimeWarning: invalid value encountered in log10
  tab['mag_auto'] = 23.9-2.5*np.log10(total_flux)
2022-12-02 01:27:53,377 - stpipe - WARNING - VerifyWarning: Keyword name 'FILTER_TYPE' is greater than 8 characters or contains characters not allowed by the FITS standard; a HIERARCH card will be created.
2022-12-02 01:27:53,380 - stpipe - WARNING - VerifyWarning: Keyword name 'THRESHOLD' is greater than 8 characters or contains characters not allowed by the FITS standard; a HIERARCH card will be created.
2022-12-02 01:27:53,384 - stpipe - WARNING - VerifyWarning: Keyword name 'ERR_SCALE' is greater than 8 characters or contains characters not allowed by the FITS standard; a HIERARCH card will be created.
2022-12

# SEP abell2744-preimg-02561-001-041.0-nrcb5-f277w-clear.cat.fits: 2664 objects
abell2744-preimg-02561-001-041.0-nrcb5-f277w-clear.cat.reg: x = X_WORLD, y=Y_WORLD, ellipse=False
# (2022-12-01 14:27:53.498)

jwst_utils.set_jwst_to_hst_keywords(**{'input': 'jw02561001002_02101_00001_nrcblong_cal.fits', 'reset': True, 'verbose': True, 'orig_keys': ['TELESCOP', 'INSTRUME', 'DETECTOR', 'FILTER', 'PUPIL', 'EXP_TYPE']})

Reset: TELESCOP > JWST (OTELESCO)
Reset: INSTRUME > NIRCAM (OINSTRUM)
Reset: DETECTOR > NRCBLONG (ODETECTO)
Reset: FILTER > F277W (OFILTER)
Reset: PUPIL > CLEAR (OPUPIL)
Reset: EXP_TYPE > NRC_IMAGE (OEXP_TYP)
# (2022-12-01 14:27:53.759)

jwst_utils.set_jwst_to_hst_keywords(**{'input': 'jw02561001002_02101_00002_nrcblong_cal.fits', 'reset': True, 'verbose': True, 'orig_keys': ['TELESCOP', 'INSTRUME', 'DETECTOR', 'FILTER', 'PUPIL', 'EXP_TYPE']})

Reset: TELESCOP > JWST (OTELESCO)
Reset: INSTRUME > NIRCAM (OINSTRUM)
Reset: DETECTOR > NRCBLONG (ODETECTO)
Reset: FILTER > F277W (OF

In [13]:
# report on failed visits 
os.chdir(os.path.join(HOME_PATH, working_dir))
if len(glob('*failed')) > 0:
    print('ERROR failed files',glob('*failed'))

# copy a wcs reference file into the working director to use as reference pixel grid
with fits.open('../hst_abell2744_f814w.fits') as ref_hdu:
    header = ref_hdu[0].header
    header['EQUINOX'] = 2000
    ref_hdu.writeto('{0}_wcs-ref.fits'.format(root), overwrite=True,output_verify='fix')


In [15]:
# finally drizzle
mosaic_args = kwargs['mosaic_args']

mosaic_args['fill_mosaics'] = False

# better 
mosaic_args['wcs_params']['pixel_scale'] = 0.06  
mosaic_args['half_optical_pixscale'] = True

# Not really ir/optical, but this comes from HST
# ir are drizzled at mosaic_args['wcs_params']['pixel_scale'],
# optical at half that

mosaic_args['ir_filters'] = ['F277W','F356W','F410M','F444W']
mosaic_args['optical_filters'] = ['F115W','F150W','F200W']

mosaic_args['wcs_params']['pad_reference'] = 6 # small padding around edge, arcsec
kwargs['mosaic_drizzle_args']['static'] = False

if True:
    auto_script.make_combined_mosaics(root, mosaic_args=mosaic_args,
                                  mosaic_drizzle_args=kwargs['mosaic_drizzle_args'])


# (2022-12-01 14:29:16.727)
{'files': ['jw02561001002_02101_00001_nrcalong_cal.fits', 'jw02561001002_02101_00002_nrcalong_cal.fits', 'jw02561001002_02101_00003_nrcalong_cal.fits', 'jw02561001002_02101_00004_nrcalong_cal.fits'], 'footprint': <shapely.geometry.polygon.Polygon object at 0x7fbf3b6f7550>, 'product': 'abell2744-preimg-02561-001-041.0-nrca5-f277w-clear'}
# (2022-12-01 14:29:16.728)
{'files': ['jw02561001002_02101_00001_nrcblong_cal.fits', 'jw02561001002_02101_00002_nrcblong_cal.fits', 'jw02561001002_02101_00003_nrcblong_cal.fits', 'jw02561001002_02101_00004_nrcblong_cal.fits'], 'footprint': <shapely.geometry.polygon.Polygon object at 0x7fbf28e63fd0>, 'product': 'abell2744-preimg-02561-001-041.0-nrcb5-f277w-clear'}
Drizzle mosaics in filters: dict_keys(['f277w'])


### drizzle_overlaps: uncover-f277w (8)

# (2022-12-01 14:29:16.728)

jwst_utils.set_jwst_to_hst_keywords(**{'input': 'jw02561001002_02101_00001_nrcalong_cal.fits', 'reset': False, 'verbose': True, 'orig_keys': ['TE

2022-12-02 01:29:19,475 - stpipe - INFO - Setting up logfile :  astrodrizzle.log


Setting up logfile :  astrodrizzle.log


2022-12-02 01:29:19,477 - stpipe - INFO - AstroDrizzle log file: astrodrizzle.log


AstroDrizzle log file: astrodrizzle.log


...

2022-12-02 01:29:33,545 - stpipe - INFO -                   Total          14.0246 sec.


                  Total          14.0246 sec.


2022-12-02 01:29:33,546 - stpipe - INFO - 


2022-12-02 01:29:33,547 - stpipe - INFO - Removing intermediate files for jw02561001002_02101_00001_nrcalong_cal.fits
2022-12-02 01:29:33,550 - stpipe - INFO - Removing intermediate files for jw02561001002_02101_00002_nrcalong_cal.fits
2022-12-02 01:29:33,553 - stpipe - INFO - Removing intermediate files for jw02561001002_02101_00003_nrcalong_cal.fits
2022-12-02 01:29:33,557 - stpipe - INFO - Removing intermediate files for jw02561001002_02101_00004_nrcalong_cal.fits
2022-12-02 01:29:33,561 - stpipe - INFO - Removing intermediate files for jw02561001002_02101_00001_nrcblong_cal.fits
2022-12-02 01:29:33,564 - stpipe - INFO - Removing intermediate files for jw02561001002_02101_00002_nrcblong_cal.fits
2022-12-02 01:29:33,568 - stpipe - INFO - Removing intermediate files for jw02561001002_02101_00003_nrcblong_cal.fits
2022-12-02 01:29:33,571 - stpipe - INFO - Removing intermediate files for jw02561001002_02101_00004_nrcblong_cal.fits
2022-12-02 01:29:33,575 - stpipe - INFO - Trailer file w

Trailer file written to:  astrodrizzle.log
# (2022-12-01 14:29:34.156)

jwst_utils.set_jwst_to_hst_keywords(**{'input': 'jw02561001002_02101_00001_nrcalong_cal.fits', 'reset': True, 'verbose': True, 'orig_keys': ['TELESCOP', 'INSTRUME', 'DETECTOR', 'FILTER', 'PUPIL', 'EXP_TYPE']})

Reset: TELESCOP > JWST (OTELESCO)
Reset: INSTRUME > NIRCAM (OINSTRUM)
Reset: DETECTOR > NRCALONG (ODETECTO)
Reset: FILTER > F277W (OFILTER)
Reset: PUPIL > CLEAR (OPUPIL)
Reset: EXP_TYPE > NRC_IMAGE (OEXP_TYP)
# (2022-12-01 14:29:34.622)

jwst_utils.set_jwst_to_hst_keywords(**{'input': 'jw02561001002_02101_00002_nrcalong_cal.fits', 'reset': True, 'verbose': True, 'orig_keys': ['TELESCOP', 'INSTRUME', 'DETECTOR', 'FILTER', 'PUPIL', 'EXP_TYPE']})

Reset: TELESCOP > JWST (OTELESCO)
Reset: INSTRUME > NIRCAM (OINSTRUM)
Reset: DETECTOR > NRCALONG (ODETECTO)
Reset: FILTER > F277W (OFILTER)
Reset: PUPIL > CLEAR (OPUPIL)
Reset: EXP_TYPE > NRC_IMAGE (OEXP_TYP)
# (2022-12-01 14:29:35.083)

jwst_utils.set_jwst_to_hst_key

In [22]:
# the reduced mosaics, mapped onto the reference grid are now in
# root_drz_sci.fits [science image]
# root_drz_sci.fits [wht image]
glob(root+'*')

['uncover_visits.yaml',
 'uncover-f277w_drz_sci.fits',
 'uncover-f277w_drz_wht.fits']

In [29]:
# inspect in ds9 

# exercise 

1) download the first 8 frames in the F150W and F444W filter
2) reduce in the same way
3) create 3 color RGB image using the F150W, F277W, F444W bands